In [1]:
import pandas as pd
import numpy as np

df = pd.read_pickle('../train/pooled_df.pkl')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'train/pooled_df.pkl'

In [ ]:
def embed(embed_string):
    # Return random embedding
    return np.random.rand(768)

def embed_data(df, embed_string):
    # Embed data
    df['code_name'] = df['code_name'].map(lambda x : [embed(embed_string) for y in x])
    return df

df = embed_data(df, 'test')
df.head()